In [90]:
import json
import requests
import traceback
from datetime import datetime, timedelta
from playwright.async_api import async_playwright
from utils.calendar_utils import (
        generate_mock_calendar,
        format_calendar_data,
        convert_to_unified_format,
        find_overlapping_times
    )
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from pprint import pprint
name = "John Doe"
email = "john@doe.com"
phone = "5109198404"

In [75]:
mock_calendar = generate_mock_calendar()
pprint(mock_calendar, indent=3)

{  'available_slots': [  {  'date': 'Friday, March 07, 2025',
                            'free_slots': [  {  'end': '02:30 PM',
                                                'start': '09:30 AM'},
                                             {  'end': '04:00 PM',
                                                'start': '03:30 PM'}]},
                         {  'date': 'Monday, March 10, 2025',
                            'free_slots': [  {  'end': '10:00 AM',
                                                'start': '09:30 AM'},
                                             {  'end': '01:00 PM',
                                                'start': '11:00 AM'},
                                             {  'end': '02:30 PM',
                                                'start': '01:30 PM'},
                                             {  'end': '04:00 PM',
                                                'start': '03:30 PM'}]},
                         {  'date': 'Tuesday, Marc

In [76]:
calendly_url = "https://calendly.com/robertjandali/30min"
url_parts = calendly_url.split('/')
profile_slug = url_parts[-2]
event_type_slug = url_parts[-1].split('?')[0]
id_url = f"https://calendly.com/api/booking/event_types/lookup?event_type_slug={event_type_slug}&profile_slug={profile_slug}"

# Get UUID
response = requests.get(id_url)
response.raise_for_status()
event_data = response.json()
uuid = event_data.get("uuid")

range_url = f"https://calendly.com/api/booking/event_types/{uuid}/calendar/range"
start_date = datetime.now()
end_date = start_date + timedelta(days=35)
params = {
    "timezone": "America/Los_Angeles",
    "diagnostics": "false",
    "range_start": start_date.strftime("%Y-%m-%d"),
    "range_end": end_date.strftime("%Y-%m-%d")
}

calendar_response = requests.get(range_url, params=params)
calendar_response.raise_for_status()
calendly_data = calendar_response.json()
calendly_formatted = format_calendar_data(calendly_data)
pprint(calendly_formatted, indent=3)

{  'available_dates': [  {  'date': 'March 07, 2025',
                            'iso_times': [  '2025-03-07T00:30:00-08:00',
                                            '2025-03-07T01:00:00-08:00',
                                            '2025-03-07T01:30:00-08:00',
                                            '2025-03-07T02:00:00-08:00',
                                            '2025-03-07T02:30:00-08:00',
                                            '2025-03-07T03:00:00-08:00',
                                            '2025-03-07T03:30:00-08:00',
                                            '2025-03-07T04:00:00-08:00',
                                            '2025-03-07T04:30:00-08:00',
                                            '2025-03-07T05:00:00-08:00',
                                            '2025-03-07T05:30:00-08:00',
                                            '2025-03-07T06:00:00-08:00',
                                            '2025-03-07T06:30:00-08:00

In [77]:
calendly_timezone = calendly_formatted.get("timezone", "America/Los_Angeles")
mock_calendar

{'week_of': 'March 07, 2025',
 'timezone': 'America/Los_Angeles',
 'available_slots': [{'date': 'Friday, March 07, 2025',
   'free_slots': [{'start': '09:30 AM', 'end': '02:30 PM'},
    {'start': '03:30 PM', 'end': '04:00 PM'}]},
  {'date': 'Monday, March 10, 2025',
   'free_slots': [{'start': '09:30 AM', 'end': '10:00 AM'},
    {'start': '11:00 AM', 'end': '01:00 PM'},
    {'start': '01:30 PM', 'end': '02:30 PM'},
    {'start': '03:30 PM', 'end': '04:00 PM'}]},
  {'date': 'Tuesday, March 11, 2025',
   'free_slots': [{'start': '09:30 AM', 'end': '10:00 AM'},
    {'start': '11:00 AM', 'end': '11:30 AM'},
    {'start': '12:00 PM', 'end': '05:00 PM'}]},
  {'date': 'Wednesday, March 12, 2025',
   'free_slots': [{'start': '09:00 AM', 'end': '10:00 AM'},
    {'start': '10:30 AM', 'end': '01:00 PM'},
    {'start': '02:00 PM', 'end': '02:30 PM'},
    {'start': '03:30 PM', 'end': '05:00 PM'}]},
  {'date': 'Thursday, March 13, 2025',
   'free_slots': [{'start': '09:00 AM', 'end': '10:00 AM'},
  

In [78]:
unified_mock = convert_to_unified_format(mock_calendar, 'mock', calendly_timezone)
unified_mock

{'timezone': 'America/Los_Angeles',
 'available_days': [{'date': 'March 07, 2025',
   'day_of_week': 'Friday',
   'available_times': ['09:30 AM',
    '10:00 AM',
    '10:30 AM',
    '11:00 AM',
    '11:30 AM',
    '12:00 PM',
    '12:30 PM',
    '01:00 PM',
    '01:30 PM',
    '02:00 PM',
    '03:30 PM']},
  {'date': 'March 10, 2025',
   'day_of_week': 'Monday',
   'available_times': ['09:30 AM',
    '11:00 AM',
    '11:30 AM',
    '12:00 PM',
    '12:30 PM',
    '01:30 PM',
    '02:00 PM',
    '03:30 PM']},
  {'date': 'March 11, 2025',
   'day_of_week': 'Tuesday',
   'available_times': ['09:30 AM',
    '11:00 AM',
    '12:00 PM',
    '12:30 PM',
    '01:00 PM',
    '01:30 PM',
    '02:00 PM',
    '02:30 PM',
    '03:00 PM',
    '03:30 PM',
    '04:00 PM',
    '04:30 PM']},
  {'date': 'March 12, 2025',
   'day_of_week': 'Wednesday',
   'available_times': ['09:00 AM',
    '09:30 AM',
    '10:30 AM',
    '11:00 AM',
    '11:30 AM',
    '12:00 PM',
    '12:30 PM',
    '02:00 PM',
    '03:

In [79]:
unified_calendly = convert_to_unified_format(calendly_formatted, 'calendly')
unified_calendly

{'timezone': 'America/Los_Angeles',
 'available_days': [{'date': 'March 07, 2025',
   'day_of_week': 'Friday',
   'available_times': ['12:30 AM',
    '01:00 AM',
    '01:30 AM',
    '02:00 AM',
    '02:30 AM',
    '03:00 AM',
    '03:30 AM',
    '04:00 AM',
    '04:30 AM',
    '05:00 AM',
    '05:30 AM',
    '06:00 AM',
    '06:30 AM',
    '07:00 AM',
    '07:30 AM',
    '08:00 AM',
    '08:30 AM',
    '01:00 PM',
    '02:00 PM',
    '02:30 PM',
    '03:00 PM',
    '03:30 PM',
    '04:00 PM',
    '04:30 PM',
    '05:00 PM',
    '05:30 PM',
    '06:00 PM',
    '06:30 PM',
    '07:00 PM',
    '07:30 PM',
    '08:00 PM',
    '08:30 PM',
    '09:00 PM',
    '09:30 PM',
    '10:00 PM',
    '10:30 PM',
    '11:00 PM']},
  {'date': 'March 10, 2025',
   'day_of_week': 'Monday',
   'available_times': ['12:00 AM',
    '12:30 AM',
    '01:00 AM',
    '01:30 AM',
    '02:00 AM',
    '02:30 AM',
    '03:00 AM',
    '03:30 AM',
    '04:00 AM',
    '04:30 AM',
    '05:00 AM',
    '05:30 AM',
    '06:

In [80]:
overlapping_calendar = find_overlapping_times(unified_mock, unified_calendly)
overlapping_calendar

{'timezone': 'America/Los_Angeles',
 'available_days': [{'date': 'March 07, 2025',
   'day_of_week': 'Friday',
   'available_times': ['01:00 PM', '02:00 PM', '03:30 PM']},
  {'date': 'March 10, 2025',
   'day_of_week': 'Monday',
   'available_times': ['09:30 AM', '01:30 PM', '02:00 PM', '03:30 PM']},
  {'date': 'March 11, 2025',
   'day_of_week': 'Tuesday',
   'available_times': ['09:30 AM',
    '11:00 AM',
    '12:00 PM',
    '01:30 PM',
    '02:00 PM',
    '02:30 PM',
    '03:00 PM',
    '03:30 PM',
    '04:00 PM',
    '04:30 PM']},
  {'date': 'March 12, 2025',
   'day_of_week': 'Wednesday',
   'available_times': ['09:00 AM',
    '09:30 AM',
    '10:30 AM',
    '11:00 AM',
    '11:30 AM',
    '12:00 PM',
    '12:30 PM',
    '02:00 PM',
    '03:30 PM',
    '04:00 PM',
    '04:30 PM']},
  {'date': 'March 13, 2025',
   'day_of_week': 'Thursday',
   'available_times': ['09:00 AM',
    '09:30 AM',
    '11:00 AM',
    '12:30 PM',
    '02:00 PM',
    '03:00 PM',
    '03:30 PM']}]}

In [81]:
from utils.calendar_utils import convert_to_unified_format, find_overlapping_times
calendly_timezone = calendly_formatted.get("timezone", "America/Los_Angeles")
unified_mock = convert_to_unified_format(mock_calendar, 'mock', calendly_timezone)
unified_calendly = convert_to_unified_format(calendly_formatted, 'calendly')
overlapping_calendar = find_overlapping_times(unified_mock, unified_calendly)
print(overlapping_calendar)

{'timezone': 'America/Los_Angeles', 'available_days': [{'date': 'March 07, 2025', 'day_of_week': 'Friday', 'available_times': ['01:00 PM', '02:00 PM', '03:30 PM']}, {'date': 'March 10, 2025', 'day_of_week': 'Monday', 'available_times': ['09:30 AM', '01:30 PM', '02:00 PM', '03:30 PM']}, {'date': 'March 11, 2025', 'day_of_week': 'Tuesday', 'available_times': ['09:30 AM', '11:00 AM', '12:00 PM', '01:30 PM', '02:00 PM', '02:30 PM', '03:00 PM', '03:30 PM', '04:00 PM', '04:30 PM']}, {'date': 'March 12, 2025', 'day_of_week': 'Wednesday', 'available_times': ['09:00 AM', '09:30 AM', '10:30 AM', '11:00 AM', '11:30 AM', '12:00 PM', '12:30 PM', '02:00 PM', '03:30 PM', '04:00 PM', '04:30 PM']}, {'date': 'March 13, 2025', 'day_of_week': 'Thursday', 'available_times': ['09:00 AM', '09:30 AM', '11:00 AM', '12:30 PM', '02:00 PM', '03:00 PM', '03:30 PM']}]}


In [82]:
response_schema = ResponseSchema(
    name="suggested_time",
    description="The suggested meeting time in ISO 8601 format",
    type="string"
)
        
        # Create the parser
parser = StructuredOutputParser.from_response_schemas([response_schema])
format_instructions = parser.get_format_instructions()

In [83]:
from prompts.scheduling_prompts import scheduling_prompt
prompt_template = scheduling_prompt(overlapping_calendar, format_instructions)
prompt_template

ChatPromptTemplate(input_variables=['format_instructions', 'overlapping_availability'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'overlapping_availability'], input_types={}, partial_variables={}, template='\n    You are a scheduling assistant. Please analyze the following available meeting times and suggest the best option.\n\n    Overlapping Available Times:\n    {overlapping_availability}\n\n    Find the best meeting time from these overlapping time slots at the earliest reasonable business hour.\n\n    {format_instructions}\n\n    Note: The suggested_time MUST be in this exact format:\n    "2025-03-05T09:30:00-08:00?month=2025-03&date=2025-03-05"\n    - Include ISO 8601 datetime with timezone offset (-08:00 for PST)\n    - Include query parameters for month and date\n    '), additional_kwargs={})])

In [84]:
# Get suggested time from LLM
messages = prompt_template.format_messages(
    overlapping_availability=json.dumps(overlapping_calendar, indent=2),
    format_instructions=format_instructions
)

In [85]:
formatted_prompt = "\n".join(message.content for message in messages)
pprint(formatted_prompt)

('\n'
 '    You are a scheduling assistant. Please analyze the following available '
 'meeting times and suggest the best option.\n'
 '\n'
 '    Overlapping Available Times:\n'
 '    {\n'
 '  "timezone": "America/Los_Angeles",\n'
 '  "available_days": [\n'
 '    {\n'
 '      "date": "March 07, 2025",\n'
 '      "day_of_week": "Friday",\n'
 '      "available_times": [\n'
 '        "01:00 PM",\n'
 '        "02:00 PM",\n'
 '        "03:30 PM"\n'
 '      ]\n'
 '    },\n'
 '    {\n'
 '      "date": "March 10, 2025",\n'
 '      "day_of_week": "Monday",\n'
 '      "available_times": [\n'
 '        "09:30 AM",\n'
 '        "01:30 PM",\n'
 '        "02:00 PM",\n'
 '        "03:30 PM"\n'
 '      ]\n'
 '    },\n'
 '    {\n'
 '      "date": "March 11, 2025",\n'
 '      "day_of_week": "Tuesday",\n'
 '      "available_times": [\n'
 '        "09:30 AM",\n'
 '        "11:00 AM",\n'
 '        "12:00 PM",\n'
 '        "01:30 PM",\n'
 '        "02:00 PM",\n'
 '        "02:30 PM",\n'
 '        "03:00 PM",

In [86]:
import os
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)
response = llm.invoke(messages)
parsed_response = parser.parse(response.content)
suggested_time = parsed_response['suggested_time']
print(f"LLM suggested time: {suggested_time}")

LLM suggested time: 2025-03-10T09:30:00-08:00?month=2025-03&date=2025-03-10


In [87]:
import pytz
from datetime import datetime
suggested_time = parsed_response['suggested_time']
clean_time_str = suggested_time.split('?')[0]
suggested_time_dt = datetime.fromisoformat(clean_time_str)
# No need for timezone conversion since it's already in PST
print(suggested_time)
print(f"LLM suggested time: {suggested_time_dt.strftime('%I:%M %p')} PST")

2025-03-10T09:30:00-08:00?month=2025-03&date=2025-03-10
LLM suggested time: 09:30 AM PST


In [88]:
iso_datetime = suggested_time.split('?')[0] if '?' in suggested_time else suggested_time
suggested_time

'2025-03-10T09:30:00-08:00?month=2025-03&date=2025-03-10'

In [89]:

base_path = '/'.join(calendly_url.split('?')[0].split('/')[:-1]) if calendly_url.endswith('/') else '/'.join(calendly_url.split('?')[0].split('/'))
final_url = f"{base_path}/{suggested_time}"
print(f"Created booking URL: {final_url}")

Created booking URL: https://calendly.com/robertjandali/30min/2025-03-10T09:30:00-08:00?month=2025-03&date=2025-03-10


In [94]:
anchor_api_key = os.getenv("ANCHOR_API_KEY")
url = "https://api.anchorbrowser.io/api/sessions"
payload = {
                    "adblock_config": {
                        "active": False,
                        "popup_blocking_active": False
                    },
                    "captcha_config": {"active": True},
                    "headless": False,
                    "proxy_config": {
                        "type": "anchor_residential",
                        "active": True
                    },
                    "recording": {"active": True},
                    "profile": {
                        "name": "my-profile",
                        "persist": True,
                        "store_cache": True
                    },
                    "viewport": {
                        "width": 1440,
                        "height": 900
                    },
                    "timeout": 10,
                    "idle_timeout": 3
                }
                

headers = {
    "anchor-api-key": anchor_api_key,
    "Content-Type": "application/json"
}

response = requests.request("POST", url, json=payload, headers=headers)
session_id = response.json()['id']
anchor_url = f"wss://connect.anchorbrowser.io?apiKey={anchor_api_key}&sessionId={session_id}"

async with async_playwright() as p:
    browser = await p.chromium.connect_over_cdp(anchor_url)
    context = browser.contexts[0]
    ai = context.service_workers[0]
    page = context.pages[0]
                        
    await page.goto(final_url)
                        
    task = f"~ Fill out the form with name {name}, email {email}, phone {phone}, and click the Schedule Event button. Wait until the confirmation page is loaded or a success message appears."
    result = await ai.evaluate(task)
    print(result)
                        
    await browser.close()

Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


Error: Worker.evaluate: SyntaxError: Unexpected identifier 'out'
    at eval (<anonymous>)
    at UtilityScript.evaluate (<anonymous>:234:30)
    at UtilityScript.<anonymous> (<anonymous>:1:44)